In [1]:
import ssms 
import lanfactory
import os
import sys
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scripts
from copy import deepcopy

importing lanfactory
importing ssms
importing hddm
importing lanfactory
importing ssms
importing hddm


In [2]:
def get_summary(model = None,
                data = None,
                show_top_n = 10):
    
    val_loss = data['val_loss']
    data = data.sort_values('val_loss')
    print('val loss mean: ', data['val_loss'].mean())
    print('val loss min: ', data['val_loss'].min())
    print('val loss max: ', data['val_loss'].max())
    
    print('Top five network architectures')
    print(data.iloc[:show_top_n, :][['val_loss', 
                            'n_hidden_layers',
                            'size_hidden_layers']])
    
    print('Top five networks, paths')
    for i in range(show_top_n):
        print(data.iloc[i, :]['filename'])
        print(data.iloc[i, :]['path'])
    return data.iloc[:show_top_n, :]

In [3]:
def find_diff_within_network_architecture(df):
    diff = df['val_loss'].max() - df['val_loss'].min()
    return diff

In [8]:
model = 'ddm'
networks_path = '/users/afengler/data/proj_ak_lan/akili/data/torch_models/lan/' + \
                        'choicep/ddm/'

In [9]:
files_ = os.listdir(networks_path)

In [10]:
# files_

In [11]:
full_dat, training_dat = scripts.make_network_df(path = networks_path,
                                                 model = model)

In [12]:
training_dat

,Unnamed: 0,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers,train_output_type,weight_decay
19,19,19.0,0.335348,7c9909f0655a11ed8ba1ac1f6bb05ad0,7c9909f0655a11ed8ba1ac1f6bb05ad0_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.01000
39,19,19.0,0.326592,6c7b945c655a11edb945ac1f6bb2f5c8,6c7b945c655a11edb945ac1f6bb2f5c8_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,120,logits,0.00100
59,19,19.0,0.347120,da90e042655911ed94c7ac1f6bb2fa9c,da90e042655911ed94c7ac1f6bb2fa9c_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,120,logits,0.01000
79,19,19.0,0.330916,750bc498655a11ed960aac1f6bb2f5c0,750bc498655a11ed960aac1f6bb2f5c0_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,100,logits,0.01000
99,19,19.0,0.445243,f60e819a656211ed99f1ac1f6bb2a38c,f60e819a656211ed99f1ac1f6bb2a38c_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,120,logits,0.10000
...,...,...,...,...,...,...,...,...,...,...,...
2579,19,19.0,0.333652,2c18470c655a11eda87bac1f6bb05b0c,2c18470c655a11eda87bac1f6bb05b0c_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,120,logits,0.00001
2599,19,19.0,0.322819,7bdb298a655a11ed95deac1f6bb2fa7c,7bdb298a655a11ed95deac1f6bb2fa7c_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,120,logits,0.00100
2619,19,19.0,0.446660,4b16ab54656311ed826eac1f6bb05b28,4b16ab54656311ed826eac1f6bb05b28_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,100,logits,0.10000
2639,19,19.0,0.329296,9a9a44ae656011edb5c8ac1f6b1b78cc,9a9a44ae656011edb5c8ac1f6b1b78cc_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,100,logits,0.00001


In [14]:
training_dat.sort_values(by = ['val_loss'])[['val_loss', 'n_hidden_layers', 'size_hidden_layers', 'weight_decay']]

,val_loss,n_hidden_layers,size_hidden_layers,weight_decay
1959,0.318677,3,100,0.00001
2079,0.319934,5,100,0.00001
1379,0.320128,4,120,0.00001
1539,0.320277,5,120,0.00000
1819,0.320358,4,100,0.00010
...,...,...,...,...
1659,0.448512,5,120,0.10000
1319,0.448877,5,100,0.10000
1839,0.449058,5,120,0.10000
1119,0.450089,4,100,0.10000


### Variability in training loss for same architectures

In [15]:
training_dat.groupby(['n_hidden_layers', 
                      'size_hidden_layers']).apply(find_diff_within_network_architecture)

n_hidden_layers  size_hidden_layers
3                100                   0.128955
                 120                   0.125001
4                100                   0.129731
                 120                   0.125980
5                100                   0.128942
                 120                   0.130747
dtype: float64

In [16]:
# Adding to Training data
training_dat = pd.merge(training_dat, training_dat.groupby(['n_hidden_layers', 
                     'size_hidden_layers']).apply(find_diff_within_network_architecture).reset_index(name = 'max_min_val_loss_diff'),
                                                                         on = ['n_hidden_layers', 'size_hidden_layers']).drop(labels = ['Unnamed: 0'], axis = 1)

In [17]:
training_dat.sort_values(by = ['n_hidden_layers', 'size_hidden_layers'])

,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers,train_output_type,weight_decay,max_min_val_loss_diff
0,19.0,0.335348,7c9909f0655a11ed8ba1ac1f6bb05ad0,7c9909f0655a11ed8ba1ac1f6bb05ad0_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.01000,0.128955
1,19.0,0.320417,7c72ebce656611ed984aac1f6b63e9f4,7c72ebce656611ed984aac1f6b63e9f4_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00001,0.128955
2,19.0,0.334163,2c07f212655a11edbf46ac1f6bb2a494,2c07f212655a11edbf46ac1f6bb2a494_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00100,0.128955
3,19.0,0.331173,839d8aa8656111ed9a550cc47afdbf59,839d8aa8656111ed9a550cc47afdbf59_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00000,0.128955
4,19.0,0.331854,8039c9b6656211ed83c8ac1f6bb2a428,8039c9b6656211ed83c8ac1f6bb2a428_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00010,0.128955
...,...,...,...,...,...,...,...,...,...,...,...
106,19.0,0.448323,e0ede1c8655a11edb7eeac1f6bb05b48,e0ede1c8655a11edb7eeac1f6bb05b48_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.10000,0.130747
107,19.0,0.322297,b8446c22657511eda619ac1f6b1b7b90,b8446c22657511eda619ac1f6b1b7b90_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.00010,0.130747
108,19.0,0.332329,dedfa98a655911ed8f65ac1f6b1b7a88,dedfa98a655911ed8f65ac1f6b1b7a88_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.00100,0.130747
109,19.0,0.322848,7ca1dd50655a11ed8c9fac1f6bb2f5b8,7ca1dd50655a11ed8c9fac1f6bb2f5b8_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.00100,0.130747


In [18]:
get_summary(model = model,
            data = deepcopy(training_dat))

val loss mean:  0.35026773832794417
val loss min:  0.3186769485473633
val loss max:  0.451023519039154
Top five network architectures
     val_loss  n_hidden_layers  size_hidden_layers
15   0.318677                3                 100
61   0.319934                5                 100
32   0.320128                4                 120
99   0.320277                5                 120
78   0.320358                4                 100
120  0.320388                3                 120
1    0.320417                3                 100
112  0.320541                3                 120
93   0.320707                5                 120
129  0.320985                3                 120
Top five networks, paths
fb2228f0656a11edb1bdac1f6b63e9f4_ddm_torch_training_history.csv
/users/afengler/data/proj_ak_lan/akili/data/torch_models/lan/choicep/ddm//fb2228f0656a11edb1bdac1f6b63e9f4_ddm_torch_training_history.csv
7dda8a44656611eda211ac1f6bb2a5d4_ddm_torch_training_history.csv
/users/afengle

,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers,train_output_type,weight_decay,max_min_val_loss_diff
15,19.0,0.318677,fb2228f0656a11edb1bdac1f6b63e9f4,fb2228f0656a11edb1bdac1f6b63e9f4_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00001,0.128955
61,19.0,0.319934,7dda8a44656611eda211ac1f6bb2a5d4,7dda8a44656611eda211ac1f6bb2a5d4_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,100,logits,0.00001,0.128942
32,19.0,0.320128,c51f0e52656611ed8648ac1f6b627e0a,c51f0e52656611ed8648ac1f6b627e0a_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,120,logits,0.00001,0.125980
99,19.0,0.320277,7c716024656611edb67aac1f6b1b7b16,7c716024656611edb67aac1f6b1b7b16_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.00000,0.130747
78,19.0,0.320358,c4fd9678656611ed9340ac1f6b1b7afc,c4fd9678656611ed9340ac1f6b1b7afc_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,4,100,logits,0.00010,0.129731
120,19.0,0.320388,286dfd3e656b11ed9043ac1f6b1b7bac,286dfd3e656b11ed9043ac1f6b1b7bac_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,120,logits,0.00010,0.125001
1,19.0,0.320417,7c72ebce656611ed984aac1f6b63e9f4,7c72ebce656611ed984aac1f6b63e9f4_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,100,logits,0.00001,0.128955
112,19.0,0.320541,0c6f8dba656a11ed958fac1f6bb2fa9c,0c6f8dba656a11ed958fac1f6bb2fa9c_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,120,logits,0.00001,0.125001
93,19.0,0.320707,6a85a1d2656a11ed9735ac1f6b1b7b16,6a85a1d2656a11ed9735ac1f6b1b7b16_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,5,120,logits,0.00000,0.130747
129,19.0,0.320985,8a943714655a11ed8f0aac1f6bb2faa8,8a943714655a11ed8f0aac1f6bb2faa8_ddm_torch_tra...,/users/afengler/data/proj_ak_lan/akili/data/to...,ddm,3,120,logits,0.00000,0.125001


In [23]:
x = pickle.load(open('data/lan_mlp/training_data_0_nbins_0_n_200000/training_data_angle_02b4386aa10d11eca75bac1f6bb2fa88.pickle', 'rb'))

In [29]:
x['data'][:, 0].max()

2.9991114

In [30]:
x['data'][:, 0].min()

-2.9988947

In [31]:
x['data'][:, 1].max()

2.999794

In [33]:
x['data'][:, 1].min()

0.30083

In [3]:
# network architecture

pickle.load(open('config_files/torch_network_train_config_ds_conflict_drift_angle_nsim_200000_dt_0.001_nps_5000_npts_2000_architecture_search.pickle', 'rb'))

{0: {'network_config': {'layer_types': ['dense', 'dense', 'dense', 'dense'],
   'layer_sizes': [100, 100, 100, 1],
   'activations': ['tanh', 'tanh', 'tanh', 'linear'],
   'loss': ['huber'],
   'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
   'model_id': 'ds_conflict_drift_angle'},
  'train_config': {'batch_size': 128,
   'n_epochs': 20,
   'optimizer': 'adam',
   'learning_rate': 0.002,
   'loss': 'huber',
   'save_history': True,
   'metrics': [<keras.losses.MeanSquaredError at 0x7f6aa401ed10>,
   'callbacks': ['checkpoint', 'earlystopping', 'reducelr'],
   'n_training_files': 1000,
   'train_val_split': 0.98,
   'cpu_batch_size': 10000,
   'gpu_batch_size': 100000,
   'shuffle_files': True,
   'label_prelog_cutoff_low': 1e-07,
   'label_prelog_cutoff_high': None},
  'training_data_folder': '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/lan_mlp/training_data_0_nbins_0_n_200000/ds_conflict_drift_angle/',
  'training_file_identifier': 'ds_conflict_drift_angle',
  